<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy

Copy folder Google Drive sang Google Drive nhanh chóng, hỗ trợ folder lớn (2000GB+)

**Tác giả:** Nguyễn Ngọc Anh Tú
- Facebook: https://www.facebook.com/NguyenNgocAnhTu.VN
- Telegram: https://t.me/NguyenNgocAnhTu

---

## ✨ Tính năng:
- ⚡ **Copy ngay** - Không scan, copy trực tiếp luôn
- 🔔 **Âm thanh** - Thông báo khi hoàn tất
- 🎯 **Exact match** - Kiểm tra file chính xác, không trùng lặp
- 🔄 **Auto-resume** - Tự tiếp tục khi timeout/disconnect
- 🔁 **Auto-retry** - Tự retry 5 lần khi rate limit
- 💾 **Checkpoint** - Lưu tiến độ mỗi 20 files + backup
- 🔍 **Filter** - Lọc theo đuôi file (.mp4, .pdf...)

---

## 📖 Cách sử dụng:

### Bước 1: Cấu hình Colab (khuyến nghị)
- Vào **Runtime** → **Change runtime type**
- Chọn **GPU: A100** và bật **High RAM** (nếu có Pro+)

### Bước 2: Chạy từng cell theo thứ tự
1. **Cell 1** - Cài đặt thư viện
2. **Cell 2** - Nhập link folder nguồn và đích
3. **Cell 3** - Chạy copy

### Bước 3: Nếu bị timeout
- Chạy lại **Cell 3** → Tool tự động resume từ chỗ dừng

---

## ⚙️ Giải thích các tùy chọn:

| Tùy chọn | Mô tả |
|----------|-------|
| **Folder đích** | Link folder Google Drive của bạn (nơi copy đến) |
| **Folder nguồn** | Link folder cần copy (có thể là Shared Drive) |
| **Bỏ qua chứa** | Bỏ qua file/folder có tên chứa text này (VD: `.tmp, backup`) |
| **Chỉ đuôi** | Chỉ copy file có đuôi này (VD: `.mp4, .pdf`) |
| **Bỏ đuôi** | Bỏ qua file có đuôi này (VD: `.tmp, .log`) |
| **Bỏ qua file đã có** | Skip file đã tồn tại ở đích (khuyến nghị BẬT) |
| **Dry-run** | Chỉ xem sẽ copy gì, không copy thật |

---

## 🔥 Recommend cho folder lớn (>500GB):
- **Colab Pro+** với **GPU A100** + **High RAM**
- Chạy ban đêm (ít rate limit)
- Nếu timeout → chạy lại, tool tự resume

In [ ]:
#@title 1️⃣ Cài đặt thư viện
!pip install tqdm -q
print("✅ Đã cài đặt xong!")

In [ ]:
#@title 2️⃣ Nhập thông tin
from ipywidgets import widgets, Layout
from IPython.display import display, HTML

style = {'description_width': '150px'}
layout = Layout(width='550px')

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='Link folder của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='Link folder cần copy', style=style, layout=layout)
exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='.tmp, backup', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ đuôi:', value='', placeholder='.pdf, .mp4', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='🚫 Bỏ đuôi:', value='', placeholder='.tmp, .log', style=style, layout=layout)
skip_exists_cb = widgets.Checkbox(description='⏭️ Bỏ qua file đã có', value=True, layout=Layout(width='250px'))
dry_run_cb = widgets.Checkbox(description='🔍 Dry-run (chỉ xem)', value=False, layout=Layout(width='250px'))

display(HTML('<b>📌 Chính:</b>'))
display(dest_text, source_text)
display(HTML('<b>🔍 Lọc:</b>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)
display(HTML('<b>🎛️ Khác:</b>'))
display(skip_exists_cb, dry_run_cb)nprint('✅ Nhập xong chạy cell tiếp!')

In [ ]:
#@title 3️⃣ Run - GDrive Turbo Copy
import os, time, re, json, logging, warnings, gc, shutil
from datetime import datetime
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from tqdm.notebook import tqdm
from IPython.display import Audio, display
import numpy as np

# Tắt warning
warnings.filterwarnings('ignore')
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)
logging.getLogger('google_auth_httplib2').setLevel(logging.ERROR)

class GDriveCopy:
    def __init__(self):
        self.total_mb = 0
        self.copied = 0
        self.skipped = 0
        self.errors = 0
        self.total_files = 0
        self.total_size_mb = 0
        self.dry_run = False
        self.skip_exists = True
        self.exclude_str = []
        self.include_ext = []
        self.exclude_ext = []
        self.done_keys = set()
        self.checkpoint = '/content/checkpoint.json'
        self.checkpoint_backup = '/content/checkpoint.json.bak'
        self.svc = None
        self.start_time = 0
        self.last_gc = 0

    def log(self, msg):
        print(f'[{datetime.now().strftime("%H:%M:%S")}] {msg}')

    def load_cp(self):
        for cp_file in [self.checkpoint, self.checkpoint_backup]:
            try:
                if os.path.exists(cp_file):
                    d = json.load(open(cp_file))
                    self.done_keys = set(d.get('k', []))
                    self.total_mb = d.get('m', 0)
                    self.copied = d.get('c', 0)
                    self.skipped = d.get('s', 0)
                    if self.done_keys:
                        self.log(f'✅ Resume: {len(self.done_keys):,} files đã xử lý ({self.total_mb/1024:.1f}GB)')
                        return True
            except Exception as e:
                self.log(f'⚠️ Checkpoint {cp_file} lỗi: {e}')
        return False

    def save_cp(self):
        try:
            data = {'k': list(self.done_keys), 'm': self.total_mb, 'c': self.copied, 's': self.skipped, 't': datetime.now().isoformat()}
            if os.path.exists(self.checkpoint):
                shutil.copy2(self.checkpoint, self.checkpoint_backup)
            with open(self.checkpoint, 'w') as f:
                json.dump(data, f)
        except Exception as e:
            self.log(f'⚠️ Save checkpoint lỗi: {e}')

    def auth(self):
        self.log('🔐 Xác thực...')
        auth.authenticate_user()
        self.svc = build('drive', 'v3')
        self.log('✅ OK')

    def play_sound(self, success=True):
        try:
            sr = 22050
            if success:
                t1 = np.linspace(0, 0.15, int(sr*0.15))
                t2 = np.linspace(0, 0.15, int(sr*0.15))
                t3 = np.linspace(0, 0.2, int(sr*0.2))
                s1 = np.sin(2*np.pi*523*t1) * np.exp(-3*t1)
                s2 = np.sin(2*np.pi*659*t2) * np.exp(-3*t2)
                s3 = np.sin(2*np.pi*784*t3) * np.exp(-3*t3)
                sound = np.concatenate([s1, s2, s3])
            else:
                t1 = np.linspace(0, 0.2, int(sr*0.2))
                t2 = np.linspace(0, 0.3, int(sr*0.3))
                s1 = np.sin(2*np.pi*400*t1) * np.exp(-2*t1)
                s2 = np.sin(2*np.pi*300*t2) * np.exp(-2*t2)
                sound = np.concatenate([s1, s2])
            display(Audio(sound, rate=sr, autoplay=True))
        except: pass

    def format_time(self, seconds):
        if seconds < 60: return f'{int(seconds)}s'
        if seconds < 3600: return f'{int(seconds//60)}m {int(seconds%60)}s'
        return f'{int(seconds//3600)}h {int((seconds%3600)//60)}m'

    def format_size(self, mb):
        if mb < 1024: return f'{mb:.1f}MB'
        return f'{mb/1024:.2f}GB'

    def get_eta(self):
        done = self.copied + self.skipped
        if done == 0 or self.total_files == 0: return '...'
        elapsed = time.time() - self.start_time
        if elapsed < 1: return '...'
        rate = done / elapsed
        remain = self.total_files - done
        if rate > 0 and remain > 0: return self.format_time(remain / rate)
        return '~'

    def get_progress(self):
        done = self.copied + self.skipped
        if self.total_files == 0: return 0
        return (done / self.total_files) * 100

    def maybe_gc(self):
        now = time.time()
        if now - self.last_gc > 60:
            gc.collect()
            self.last_gc = now

    def ok_ext(self, name):
        if not name: return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext: return ext in self.include_ext
        if self.exclude_ext: return ext not in self.exclude_ext
        return True

    def clean_name(self, name):
        if not name: return 'unnamed'
        for c in '\\/:*?"<>|': name = name.replace(c, '_')
        name = ''.join(c for c in name if ord(c) >= 32)
        return name[:200].strip() or 'unnamed'

    def call(self, fn, max_retry=5):
        for i in range(max_retry):
            try: return fn()
            except HttpError as e:
                status = e.resp.status
                if status == 404: return None
                if status in [403, 429, 500, 502, 503, 504]:
                    wait = min(3 * (2**i), 60)
                    self.log(f'⚠️ API {status}, retry {i+1}/{max_retry} sau {wait}s...')
                    time.sleep(wait)
                else: raise
            except Exception as e:
                if i < max_retry - 1:
                    wait = min(2 * (2**i), 30)
                    time.sleep(wait)
                else: raise
        return None

    def ls(self, fid):
        items, token = [], None
        q = f"'{fid}' in parents and trashed=false"
        for s in self.exclude_str:
            s_esc = s.replace("'", "\\'")
            q += f" and not name contains '{s_esc}'"
        while True:
            r = self.call(lambda: self.svc.files().list(
                q=q, pageToken=token, pageSize=1000,
                fields='files(id,name,mimeType,size),nextPageToken',
                supportsAllDrives=True, includeItemsFromAllDrives=True
            ).execute())
            if not r: break
            for f in r.get('files', []):
                m = f.get('mimeType', '')
                if m == 'application/vnd.google-apps.folder' or (self.ok_ext(f.get('name')) and not m.startswith('application/vnd.google-apps.')):
                    items.append(f)
            token = r.get('nextPageToken')
            if not token: break
        return items

    def count_files(self, fid, depth=0, path=''):
        count, size_mb = 0, 0
        items = self.ls(fid)
        for f in items:
            if f.get('mimeType') == 'application/vnd.google-apps.folder':
                fname = f.get('name', 'unnamed')
                if depth < 2: self.log(f'  📂 Scanning: {path}/{fname}')
                c, s = self.count_files(f['id'], depth+1, f'{path}/{fname}')
                count += c
                size_mb += s
            else:
                count += 1
                size_mb += int(f.get('size', 0)) / (1024*1024)
        if depth == 0 or count % 1000 == 0: self.maybe_gc()
        return count, size_mb

    def exists(self, did, name):
        if not self.skip_exists: return None
        try:
            esc = name.replace("'", "\\'")
            r = self.call(lambda: self.svc.files().list(
                q=f"'{did}' in parents and name='{esc}' and trashed=false",
                fields='files(id)', supportsAllDrives=True, pageSize=1
            ).execute())
            return r['files'][0]['id'] if r and r.get('files') else None
        except: return None

    def cp(self, did, src):
        fid = src['id']
        name = self.clean_name(src.get('name', 'unnamed'))
        size = int(src.get('size', 0))
        mb = size / (1024*1024)
        key = f"{did}:{fid}"
        if key in self.done_keys:
            self.skipped += 1
            return
        if self.dry_run:
            self.log(f'[DRY] {name} ({mb:.1f}MB)')
            self.total_mb += mb
            self.copied += 1
            return
        if self.skip_exists and self.exists(did, name):
            self.skipped += 1
            self.done_keys.add(key)
            return
        try:
            t0 = time.time()
            r = self.call(lambda: self.svc.files().copy(
                body={'parents': [did], 'name': name},
                fileId=fid, supportsAllDrives=True
            ).execute(), max_retry=5)
            if r:
                elapsed = max(time.time() - t0, 0.1)
                spd = mb / elapsed
                self.total_mb += mb
                self.copied += 1
                self.done_keys.add(key)
                pct = self.get_progress()
                eta = self.get_eta()
                self.log(f'✅ [{self.copied}] {name[:40]} ({mb:.1f}MB, {spd:.0f}MB/s)')
                self.maybe_gc()
        except Exception as e:
            self.errors += 1
            self.log(f'❌ {name[:40]}: {str(e)[:50]}')

    def mkdir(self, did, name):
        name = self.clean_name(name)
        eid = self.exists(did, name)
        if eid: return eid
        if self.dry_run: return f'dry_{name}'
        try:
            r = self.call(lambda: self.svc.files().create(
                body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [did]},
                fields='id', supportsAllDrives=True
            ).execute())
            if r:
                self.log(f'📁 {name}')
                return r['id']
        except Exception as e:
            self.log(f'❌ mkdir {name}: {e}')
        return None

    def copy_dir(self, did, sid, depth=0):
        items = self.ls(sid)
        if not items: return
        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']
        self.log(f'{"  "*depth}📊 {len(files)} files, {len(folders)} folders')
        if files:
            for i, f in enumerate(files):
                self.cp(did, f)
                if (i+1) % 20 == 0: self.save_cp()
            self.save_cp()
        for folder in folders:
            fname = folder.get('name', 'unnamed')
            self.log(f'{"  "*depth}📁 {fname}')
            sub = self.mkdir(did, fname)
            if sub and not sub.startswith('dry_'): self.copy_dir(sub, folder['id'], depth+1)

    def get_id(self, url):
        if not url: return None
        m = re.search(r'[-\w]{25,}', url)
        return m.group(0) if m else None

    def run(self, dest, src):
        self.start_time = time.time()
        self.last_gc = time.time()
        self.log('='*50)
        self.log('🚀 GDRIVE TURBO COPY')
        self.log('='*50)
        self.auth()
        self.load_cp()
        did, sid = self.get_id(dest), self.get_id(src)
        if not did or not sid:
            self.log('❌ URL không hợp lệ!')
            return
        try:
            si = self.svc.files().get(fileId=sid, supportsAllDrives=True).execute()
            di = self.svc.files().get(fileId=did, supportsAllDrives=True).execute()
        except Exception as e:
            self.log(f'❌ Không truy cập được: {e}')
            return
        self.log(f'📂 Nguồn: {si.get("name")}')
        self.log(f'📂 Đích: {di.get("name")}')
        self.log('-'*50)
        new_dest = self.mkdir(did, si.get('name', 'Copy'))
        if not new_dest:
            self.log('❌ Không tạo được folder đích!')
            self.play_sound(False)
            return
        self.copy_dir(new_dest, sid)
        self.save_cp()
        elapsed = time.time() - self.start_time
        gb = self.total_mb / 1024
        spd = self.total_mb / max(elapsed, 1)
        self.log('='*50)
        self.log(f'✅ Copied: {self.copied:,} | ⏭️ Skipped: {self.skipped:,} | ❌ Errors: {self.errors}')
        self.log(f'💾 {gb:.2f}GB | ⏱️ {self.format_time(elapsed)} | 🚀 {spd:.1f}MB/s')
        self.log('='*50)
        if self.errors == 0:
            self.log('🎉 HOÀN TẤT!')
            self.play_sound(True)
        else:
            self.log(f'⚠️ Có {self.errors} lỗi. Chạy lại để retry.')
            self.play_sound(False)

print("🔧 Khởi tạo...")
dl = GDriveCopy()
dl.exclude_str = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [('.' + s.strip().lstrip('.').lower()) for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [('.' + s.strip().lstrip('.').lower()) for s in exclude_ext_text.value.split(',') if s.strip()]
dl.skip_exists = skip_exists_cb.value
dl.dry_run = dry_run_cb.value
if not dest_text.value.strip(): print("❌ Nhập folder đích!")
elif not source_text.value.strip(): print("❌ Nhập folder nguồn!")
else: dl.run(dest_text.value.strip(), source_text.value.strip())

In [ ]:
#@title 4️⃣ Xóa checkpoint (chạy lại từ đầu)
import os
for f in ['/content/checkpoint.json', '/content/checkpoint.json.bak']:
    if os.path.exists(f): os.remove(f); print(f'✅ Đã xóa {f}')
print('🔄 Có thể chạy lại từ đầu!')